In [9]:
import openai
import pandas as pd
import requests
import os
from io import BytesIO
from openai import OpenAI
import concurrent.futures
import base64
import time

In [10]:
df=pd.read_csv('data_detection_model_input.csv')
df=df.drop(columns=df.columns[0])
df

,accommodation_id,longitude,latitude,driving_duration,walking_duration,transit_duration,image_id,image,tag_name,probability,accommodation_name,dt,distance_to_bigben,description,score,response_time
0,1504047,-0.12759,51.52702,13,33,19,DX64WCVN23LSWFWPBOWT5YGBMO45N3SF3HBQLCFHXQUJCV...,https://imgcy.trivago.com/partner-images/1d/fd...,building,0.9999,Studios 2 Let Serviced Apartments,2024-03-07,1.998302,NaN,0.750398,0.124018
1,10405908,-0.12750,51.50722,4,14,14,RB77EISJ5MQDUEGFZSGFH3EGQQNEDPHANSIYFNAFJJKIN6...,https://imgcy.trivago.com/partner-images/88/7f...,building,1.0000,Lovely Family Apartments For Upto 6 Near Oxfor...,2024-03-07,0.820251,NaN,0.413374,0.468961
2,18426,-0.18294,51.49556,21,77,17,SWPQ2FXDSA2PH4KDMR56KNBPFJZ5C77DBKGBBAA4BPG5CG...,https://imgcy.trivago.com/partner-images/95/9f...,building,1.0000,Meliá London Kensington Meliá Collection,2024-03-07,4.866410,NaN,0.632710,0.451684
3,22032936,-0.12750,51.50722,4,14,11,3DL2RXNO4NVWZFBC4L2YATHITDKB2DDWBF5LQ6JLO2C6GM...,https://imgcy.trivago.com/partner-images/d8/d7...,building,1.0000,Victorian Conversion Garden Flat,2024-03-07,0.820251,NaN,0.579433,0.213040
4,20648498,-0.12750,51.50722,4,14,9,4MLPB64JFHPSIM2YTAFY4U5HCCVUIUABRH7GUVMVTJZKKO...,https://imgcy.trivago.com/partner-images/e3/16...,building,1.0000,Stunning Central Richmond With Garden,2024-03-07,0.820251,NaN,0.650984,0.381180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,46832,-0.10219,51.50610,9,23,16,3ERUOOVFDLAGJLMBZHU22QRWRFUPS4ZQT36ZWD5XMIAHSX...,https://imgcy.trivago.com/partner-images/d9/23...,building,0.9999,Holiday Inn Express London - Southwark,2024-03-07,1.114754,Palace Of Westminster,0.376706,0.361036
912,46539,-0.07737,51.51093,9,40,14,HHWELDHJZU6GDBQT64L4BQMRMQFN24R4VHKAV4NLWPXD6T...,https://imgcy.trivago.com/partner-images/39/ec...,building,0.9999,Bridgestreet Pepys Street,2024-03-07,2.727606,Palace Of Westminster,0.465586,0.290676
913,46461,-0.13245,51.50752,10,19,11,YOCE5VDTKT3X6YCMRVWEMD36FJL2MK3AVYRMUZSBB46FFL...,https://imgcy.trivago.com/partner-images/c3/84...,building,0.9972,Sofitel London St James,2024-03-07,1.129889,Palace Of Westminster,0.708477,0.402796
914,8003044,-0.09626,51.51041,5,21,14,GXVVATZDDOWNRD5CN7CUVAE7XGOJ3JNLY5H64EMT2RX7TA...,https://imgcy.trivago.com/partner-images/35/eb...,building,0.9999,Superior 2 Bedroom 2 Bath Apartment St Pauls L...,2024-03-07,1.419610,Palace Of Westminster,0.671675,0.462791


In [11]:
# OpenAI API Key
api_key = ""

In [13]:
# Directory for downloaded images
download_dir = "images"

if not os.path.exists(download_dir):
    os.makedirs(download_dir)

def download_image(url, path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(path, 'wb') as f:
            f.write(response.content)

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def get_image_description(base64_image, api_key):
    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }
    payload = {
      "model": "gpt-4-vision-preview",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": "Which Landmark is in the image? Indicate only one of the names of the landmarks from this list: Tower Bridge, The British Museum, Big Ben, Lastminute.com London Eye, Tower Of London, Trafalgar Square, Westminster Bridge, 30 St Mary Axe (The Gherkin), Palace Of Westminster, The Shard, Wembley Stadium, None"
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
              }
            }
          ]
        }
      ],
      "max_tokens": 20
    }
    start_time = time.time()  # Start timing
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    end_time = time.time()
    response_time = end_time - start_time
    if response.status_code == 200:
        description = response.json()['choices'][0]['message']['content']
        return description, response_time
    else:
        return "Error in API call", response_time


In [14]:
### from this list: Tower Bridge, The British Museum, Big Ben, Lastminute.com London Eye, Tower Of London, Trafalgar Square, Westminster Bridge, 30 St Mary Axe (The Gherkin), Palace Of Westminster, The Shard, Wembley Stadium, None"
     

In [15]:
#Add a new column for detections
df['response_time_vision'] = 0.0

for index, row in df.iterrows():
    image_id = row['image_id']
    image_path = row['image']
    local_path = os.path.join(download_dir, f"{image_id}.jpg")
    
    # Download the image
    download_image(image_path, local_path)
    
    # Encode the image
    base64_image = encode_image(local_path)
    
    # Get the image description from the Vision API
    description, response_time = get_image_description(base64_image, api_key)
    
    # Here you would save the description if needed, and save the response time
    df.at[index, 'response_time_vision'] = response_time
    
    # Update the DataFrame
    df.at[index, 'detections_vision_openai'] = description


In [16]:
df

,accommodation_id,longitude,latitude,driving_duration,walking_duration,transit_duration,image_id,image,tag_name,probability,accommodation_name,dt,distance_to_bigben,description,score,response_time,response_time_vision,detections_vision_openai
0,1504047,-0.12759,51.52702,13,33,19,DX64WCVN23LSWFWPBOWT5YGBMO45N3SF3HBQLCFHXQUJCV...,https://imgcy.trivago.com/partner-images/1d/fd...,building,0.9999,Studios 2 Let Serviced Apartments,2024-03-07,1.998302,NaN,0.750398,0.124018,3.288819,None
1,10405908,-0.12750,51.50722,4,14,14,RB77EISJ5MQDUEGFZSGFH3EGQQNEDPHANSIYFNAFJJKIN6...,https://imgcy.trivago.com/partner-images/88/7f...,building,1.0000,Lovely Family Apartments For Upto 6 Near Oxfor...,2024-03-07,0.820251,NaN,0.413374,0.468961,4.299603,None
2,18426,-0.18294,51.49556,21,77,17,SWPQ2FXDSA2PH4KDMR56KNBPFJZ5C77DBKGBBAA4BPG5CG...,https://imgcy.trivago.com/partner-images/95/9f...,building,1.0000,Meliá London Kensington Meliá Collection,2024-03-07,4.866410,NaN,0.632710,0.451684,5.174545,None
3,22032936,-0.12750,51.50722,4,14,11,3DL2RXNO4NVWZFBC4L2YATHITDKB2DDWBF5LQ6JLO2C6GM...,https://imgcy.trivago.com/partner-images/d8/d7...,building,1.0000,Victorian Conversion Garden Flat,2024-03-07,0.820251,NaN,0.579433,0.213040,2.592985,The Shard
4,20648498,-0.12750,51.50722,4,14,9,4MLPB64JFHPSIM2YTAFY4U5HCCVUIUABRH7GUVMVTJZKKO...,https://imgcy.trivago.com/partner-images/e3/16...,building,1.0000,Stunning Central Richmond With Garden,2024-03-07,0.820251,NaN,0.650984,0.381180,4.568479,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,46832,-0.10219,51.50610,9,23,16,3ERUOOVFDLAGJLMBZHU22QRWRFUPS4ZQT36ZWD5XMIAHSX...,https://imgcy.trivago.com/partner-images/d9/23...,building,0.9999,Holiday Inn Express London - Southwark,2024-03-07,1.114754,Palace Of Westminster,0.376706,0.361036,4.262479,Palace Of Westminster
912,46539,-0.07737,51.51093,9,40,14,HHWELDHJZU6GDBQT64L4BQMRMQFN24R4VHKAV4NLWPXD6T...,https://imgcy.trivago.com/partner-images/39/ec...,building,0.9999,Bridgestreet Pepys Street,2024-03-07,2.727606,Palace Of Westminster,0.465586,0.290676,3.912997,Big Ben
913,46461,-0.13245,51.50752,10,19,11,YOCE5VDTKT3X6YCMRVWEMD36FJL2MK3AVYRMUZSBB46FFL...,https://imgcy.trivago.com/partner-images/c3/84...,building,0.9972,Sofitel London St James,2024-03-07,1.129889,Palace Of Westminster,0.708477,0.402796,4.163838,The landmark in the image is the Palace of Wes...
914,8003044,-0.09626,51.51041,5,21,14,GXVVATZDDOWNRD5CN7CUVAE7XGOJ3JNLY5H64EMT2RX7TA...,https://imgcy.trivago.com/partner-images/35/eb...,building,0.9999,Superior 2 Bedroom 2 Bath Apartment St Pauls L...,2024-03-07,1.419610,Palace Of Westminster,0.671675,0.462791,6.533917,Big Ben


In [17]:
df.to_csv('detection_results_vision_openai.csv', index=False)